# Features Extraction Function

In [1]:
import numpy as np
import pandas as pd

HELPERGETFEATURES function accepts the raw measurement data and computes
the following feature:

-Q_{100-10}(V) variance [DeltaQ_var]                                                                                           
-Q_{100-10}(V) minimum [DeltaQ_min]                                                                                            
-Slope of linear fit to the capacity fade curve cycles 2 to 100 [CapFadeCycle2Slope]                                           
-Intercept of linear fit to capacity fade curve, cycles 2 to 100 [CapFadeCycle2Intercept]                                       
-Discharge capacity at cycle 2 [Qd2]                                                                                           
-Average charge time over first 5 cycles [AvgChargeTime]                                                                       
-Minimum Internal Resistance, cycles 2 to 100 [MinIR]                                                                          
-Internal Resistance difference between cycles 2 and 100 [IRDiff2And100]

In [2]:
def Feature(batch,f):

    #First covnert Matlab .mat files to Python dictionary
    num_cells = batch['summary'].shape[0]
    batch_dict = {}
    for i in range(num_cells):
        cl = f[(batch['cycle_life'][i,0])]
        Vdlin = np.hstack(f[batch['Vdlin'][i,0]][0,:].tolist())
        summary_CY = np.hstack(f[batch['summary'][i,0]]['cycle'][0,:].tolist())
        summary_QD = np.hstack(f[batch['summary'][i,0]]['QDischarge'][0,:].tolist())
        summary_IR = np.hstack(f[batch['summary'][i,0]]['IR'][0,:].tolist())
        summary_CT = np.hstack(f[batch['summary'][i,0]]['chargetime'][0,:].tolist())
        summary = {'cycle': summary_CY,'QD': summary_QD,'IR': summary_IR, 'chargetime': summary_CT}

        cycles = f[batch['cycles'][i,0]]
        cycle_dict = {}
        for j in range(cycles['t'].shape[0]):
            t = np.hstack(f[cycles['t'][j,0]])
            T = np.hstack(f[cycles['T'][j,0]])
            Qdlin = np.hstack(f[cycles['Qdlin'][j,0]])

            cd = {'t':t,'T': T,'Qdlin': Qdlin}
            cycle_dict[str(j)] = cd


        cell_dict = {'cycle_life': cl, 'Vdlin':Vdlin, 'summary': summary, 'cycles': cycle_dict}
        key = 'c' + str(i)
        batch_dict[key]=   cell_dict    
    #return batch_dict

    #Then extract useful features from the dictionary
    DeltaQ_var=[]
    DeltaQ_min=[]
    CapFadeCycle2Slope=[]
    CapFadeCycle2Intercept=[]
    Qd2=[]
    AvgChargeTime=[]
    MinIR=[]
    IRDiff2And100=[]
    CycleLife=[]
    for i in batch_dict.keys():    
        cl = batch_dict[i]['cycle_life'][0,0]
        #compute Q_100_10 stats
        a = np.log10(abs((batch_dict[i]['cycles']['100']['Qdlin']-batch_dict[i]['cycles']['10']['Qdlin']).var()))
        b = np.log10(abs((batch_dict[i]['cycles']['100']['Qdlin']-batch_dict[i]['cycles']['10']['Qdlin']).min()))

        # Slope and intercept of linear fit for capacity fade curve from cycle 2 to cycle 100
        coeff2=np.polyfit(batch_dict[i]['summary']['cycle'][2:100], batch_dict[i]['summary']['QD'][2:100],1)
        c = coeff2[0]
        d = coeff2[1]

        #  Discharge capacity at cycle 2
        e=batch_dict[i]['summary']['QD'][2]

        # Avg charge time, first 5 cycles (2 to 6)
        f=np.mean(batch_dict[i]['summary']['chargetime'][2:6])

        #minimum iR from cycles 2 to 100
        g=min(x for x in batch_dict[i]['summary']['IR'][2:100] if x!=0)
        h= batch_dict[i]['summary']['IR'][100]-batch_dict[i]['summary']['IR'][2]
        CycleLife.append(cl)
        DeltaQ_var.append(a)
        DeltaQ_min.append(b)
        CapFadeCycle2Slope.append(c)
        CapFadeCycle2Intercept.append(d)
        Qd2.append(e)
        AvgChargeTime.append(f)
        MinIR.append(g)
        IRDiff2And100.append(h)
        j={"DeltaQ_var":DeltaQ_var,"DeltaQ_min":DeltaQ_min,"CapFadeCycle2Slope":CapFadeCycle2Slope,
           "CapFadeCycle2Intercept":CapFadeCycle2Intercept,"Qd2":Qd2,"AvgChargeTime":AvgChargeTime,
           "MinIR":MinIR,"IRDiff2And100":IRDiff2And100,"CycleLife":CycleLife}
    df=pd.DataFrame(j)
    return batch_dict, df